## Colour Map API Examples

This notebook demonstrates how to work with Evo Colour Maps using **direct API calls**, providing granular control and detailed insight into the underlying service interactions.

### API vs SDK Approach

This notebook uses **direct Colour Map API calls** which:
- ✅ Provides full control over API requests and responses
- ✅ Shows detailed API response structures and metadata
- ✅ Allows for advanced customization and fine-tuning
- ✅ Helps understand the underlying service architecture
- ✅ Useful for debugging and advanced use cases

### Want a Simpler, High-Level Interface?

If you prefer a more streamlined experience with less boilerplate code, check out the `sdk-examples.ipynb` notebook in this same directory. The SDK examples show:
- Simplified method calls using `ColormapAPIClient`
- Automatic handling of complex API interactions
- Better error handling and validation
- Recommended for most common use cases

In [ ]:
from evo.notebooks import ServiceManagerWidget

# Evo app credentials
client_id = "<your-client-id>"  # Replace with your client ID
redirect_url = "<your-redirect-url>"  # Replace with your redirect URL

manager = await ServiceManagerWidget.with_auth_code(
    redirect_url=redirect_url,
    client_id=client_id,
).login()

### Prepare Evo SDK parameters

In [ ]:
# Get the environment and connector from the ServiceManagerWidget instance.
# The environment contains the hub URL, organization ID, and workspace ID.
# The connector is used to make API calls to the Evo service.
environment = manager.get_environment()
connector = manager.get_connector()

# Copy the environment details to local variables for easier access.
evo_hub_url = environment.hub_url
org_id = environment.org_id
workspace_id = environment.workspace_id

### List all objects in the workspace

List all objects in the workspace using direct API calls.

In [ ]:
import json
from http import HTTPStatus

import pandas as pd
from prettytable import PrettyTable

from evo.common.data import HTTPResponse

try:
    resource_path = "/geoscience-object/orgs/{org_id}/workspaces/{workspace_id}/objects"

    path_params = {
        "org_id": org_id,
        "workspace_id": workspace_id,
    }

    api_response = await connector.call_api(
        method="GET",
        resource_path=resource_path,
        path_params=path_params,
        response_types_map={
            "200": HTTPResponse,
        },
    )

    status = api_response.status
    if status != HTTPStatus.OK:
        raise RuntimeError(f"Error: Failed to list objects. Status: {status}")

    # Parse the response data
    response = api_response.data.decode("utf-8")
    response_json = json.loads(response)

    # Display as a table
    table = PrettyTable(["Name", "Object ID"])
    for obj in response_json.get("objects", []):
        table.add_row([obj.get("name", "").ljust(20), str(obj.get("object_id", "")).ljust(40)])

    if len(table.rows) == 0:
        print("No objects found.")
    else:
        print(table)

except Exception as e:
    print(f"Error listing objects:\n{e}")

### Get colour map associations for an object

Enter the `Object ID` value for the chosen object in the cell below, then retrieve its colour map associations.

In [ ]:
import json
from http import HTTPStatus

from pygments import highlight
from pygments.formatters import TerminalTrueColorFormatter
from pygments.lexers import JsonLexer

from evo.common.data import HTTPResponse

object_id = "<your-object-id>"  # Replace with actual object ID

try:
    resource_path = "/colormap/orgs/{org_id}/workspaces/{workspace_id}/objects/{object_id}/associations"

    path_params = {
        "org_id": org_id,
        "workspace_id": workspace_id,
        "object_id": object_id,
    }

    api_response = await connector.call_api(
        method="GET",
        resource_path=resource_path,
        path_params=path_params,
        response_types_map={
            "200": HTTPResponse,
        },
    )

    status = api_response.status
    if status != HTTPStatus.OK:
        raise RuntimeError(f"Error: Failed to get associations. Status: {status}")

    # Parse the response data
    response = api_response.data.decode("utf-8")
    response_json = json.loads(response)

    # Print the response in a highlighted format
    print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

    # Display as a table
    associations = response_json.get("associations", []) or response_json.get("items", [])
    association_list = [
        {
            "Colormap ID": assoc.get("colormap_id"),
            "Object ID": object_id,
            "Created At": assoc.get("created_at"),
        }
        for assoc in associations
    ]

    df = pd.DataFrame(association_list)
    display(df)

except Exception as e:
    print(f"Error getting associations:\n{e}")

### Fetch colour map metadata

Enter the `Colormap ID` value for the chosen colour map in the cell below.

In [ ]:
import json
from http import HTTPStatus

import matplotlib.pyplot as plt
import numpy as np

from evo.common.data import HTTPResponse

colormap_id = "b6e0778a-66bb-4f19-bb2d-c04af86f1bca"  # Replace with actual colormap ID

try:
    resource_path = "/colormap/orgs/{org_id}/workspaces/{workspace_id}/colormaps/{colormap_id}"

    path_params = {
        "org_id": org_id,
        "workspace_id": workspace_id,
        "colormap_id": colormap_id,
    }

    api_response = await connector.call_api(
        method="GET",
        resource_path=resource_path,
        path_params=path_params,
        response_types_map={
            "200": HTTPResponse,
        },
    )

    status = api_response.status
    if status != HTTPStatus.OK:
        raise RuntimeError(f"Error: Failed to get colormap. Status: {status}")

    # Parse the response data
    response = api_response.data.decode("utf-8")
    response_json = json.loads(response)

    # Extract and display colour map details
    attribute_controls = response_json.get("attribute_controls", [])
    gradient_controls = response_json.get("gradient_controls", [])
    colors = response_json.get("colors", [])
    map = response_json.get("map", [])

    # Check if map is empty or not
    if map:
        # If map is not empty, print and render map-color pairs
        print("\nCategory-to-Color Mapping:")

        # Find the longest category name for alignment
        max_category_length = max(len(str(category)) for category in map) if map else 0

        for category, color in zip(map, colors):
            category_str = str(category).ljust(max_category_length)
            print(f"  {category_str} → RGB{tuple(color)}")

        # Create visual color swatches
        print("\nColor Visualization:")
        fig, axes = plt.subplots(1, len(colors), figsize=(min(len(colors) * 1.5, 15), 2))
        if len(colors) == 1:
            axes = [axes]

        for idx, (category, color) in enumerate(zip(map, colors)):
            # Normalize RGB values from 0-255 to 0-1 for matplotlib
            normalized_color = [c / 255.0 for c in color]
            axes[idx].add_patch(plt.Rectangle((0, 0), 1, 1, facecolor=normalized_color))
            axes[idx].set_xlim(0, 1)
            axes[idx].set_ylim(0, 1)
            axes[idx].axis("off")
            axes[idx].set_title(str(category), fontsize=10, pad=10)

        plt.tight_layout()
        plt.show()
    else:
        # If map is empty, print attribute controls, gradient controls, and colors

        print(f"\nAttribute controls: {attribute_controls}")
        print(f"\nGradient controls: {gradient_controls}")

        print("\nRGB colors:")
        for color in colors:
            print(f"  {color}")

        # Create visual gradient/color bar
        if colors:
            print("\nColor Visualization:")
            fig, ax = plt.subplots(1, 1, figsize=(10, 1))

            # Normalize RGB values from 0-255 to 0-1
            normalized_colors = [[c / 255.0 for c in color] for color in colors]

            # Create gradient
            gradient = np.linspace(0, 1, len(colors))
            gradient = np.vstack((gradient, gradient))

            ax.imshow(gradient, aspect="auto", cmap=plt.matplotlib.colors.ListedColormap(normalized_colors))
            ax.set_yticks([])
            ax.set_xticks([])
            ax.set_title("Color Gradient")

            plt.tight_layout()
            plt.show()

except Exception as e:
    print(f"Error fetching colormap:\n{e}")